In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import requests
import json
import os

GITHUB_URL = "https://api.github.com/"

"""
Function to get JSON response from a URL
:params:
    url     string
:return:
            JSON
"""
def __get_json_response(url):
    headers = {'Authorization': 'token ' + os.environ['GITHUB_API_KEY']}
    response = requests.get(url, headers=headers)
    return json.loads(response.text)

"""
Function to add email to a set of emails and set a loop break flag
:params:
    email_set       set         set of all the emails for the user
    email           string      new email to be added
    max_len         integer     maximum number of emails to be extracted
:return:
    email_set       set         set of all the emails for the user
    break_flag      boolean     if max_limit is reached, break_flag is set to True
"""
def __add_email(email_set, email, max_len):
    email_set = email_set | set([email])
    break_flag = (len(email_set) >= max_len)

    return email_set, break_flag

"""
Function to get user emails using GitHub APIs
:params:
    user        string      a valid GitHub username
    max_limit   integer     maximum number of email ID to be fetched
:return:
    user_email  set         a set of all emails extracted
    message     string      if any error occurs, this holds the respective error message
"""
def __get_github_emails(user, max_limit):
    user_email = set([])
    break_flag = False
    try:
        users_profile_url = GITHUB_URL + "users/{0}".format(user)
        response = __get_json_response(users_profile_url)

        # some error encountered
        if 'message' in response:
            if response['message'] == 'Not Found':
                return u'You need to enter a valid GitHub Username'
            else:
                return response['message']

        name = response['name']
        user_name = response['name']

        # if user has a public email, add that to the set of emails
        if response['email']:
            user_email, break_flag = __add_email(user_email, response['email'], max_limit)

        if not break_flag:
            users_repository_url = GITHUB_URL + "users/{0}/repos?type=owner&sort=updated".format(user)
            response = __get_json_response(users_repository_url)

            for repo in response:
                if not repo['fork']:
                    users_repository_name = repo['full_name']
                    repos_commit_url = GITHUB_URL + "repos/{0}/commits".format(users_repository_name)
                    commit_reponse = __get_json_response(repos_commit_url)
                    
                    possible_positions = ['committer', 'author']

                    for commit in commit_reponse:
                        for i in possible_positions:
                            if commit['commit'][i]['name'] == user_name:
                                email_string = commit['commit'][i]['email']
                                if "noreply" not in email_string:
                                    user_email, break_flag = __add_email(user_email, email_string, max_limit)

                        if break_flag:
                            break

                if break_flag:
                    break

        if len(user_email) > 0:
            return user_email, name
        else:
            return u'No emails found', u'No first name found'

    except requests.exceptions.ConnectionError:
        return u'Proper internet connection not found', u'Proper internet connection not found'

"""
Function to get the emails associated to a username on GitHub
:params:
    username    string      a valid GitHub username
    num         integer     maximum number of email ID to be fetched, default 1
:return:
    response    JSON response
        success     boolean     flag to determine other key in JSON
        email       list        if 'success' is True, list of all the emails fetched
        message     string      if 'success' is False, returns the error message
"""
def get(username, num=1):
    github_email_response, github_full_name = __get_github_emails(username, num)
    
    if type(github_email_response) == set:
        response = {
            'success' : True,
            'data': [github_email_response, github_full_name]
        }
    else:
        response = {
            'success' : False,
            'message' : [github_email_response, github_full_name]
        }

    return response

In [103]:
def get_email_content(first_name, issue_url):
    return '''
    <!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN" "http://www.w3.org/TR/REC-html40/loose.dtd">
    <html xmlns="http://www.w3.org/1999/xhtml" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office">

    <head>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
      <!--[if gte mso 15]>
      <xml>
        <o:OfficeDocumentSettings>
        <o:AllowPNG/>
        <o:PixelsPerInch>96</o:PixelsPerInch>
        </o:OfficeDocumentSettings>
      </xml>
      <![endif]-->
      <meta charset="UTF-8">
      <meta http-equiv="X-UA-Compatible" content="IE=edge">
      <meta name="viewport" content="width=device-width, initial-scale=1">



      <title>Speed</title>

      <!-- hide media queries from Outlook -->
      <!--[if !mso]><!-->
      <style type="text/css" data-premailer="ignore">
        .apple-link-overwrite-gray a {
          color: #A8ACB1 !important;
          text-decoration: none;
        }
        .apple-link-overwrite-black a {
          color: #777A81 !important;
          text-decoration: none;
        }
        .apple-link-overwrite-black2 a {
          color: #181A1D !important;
          text-decoration: none;
        }
         .apple-link-overwrite-white a {
          color: #ffffff !important;
          text-decoration: none;
        }
        .ns span, .ns a{color:#777A81 !important; text-decoration:none !important; border:none !important;}
        @media only screen and (min-width:768px) {
          .template-container {
            width: 600px !important;
          }
        }

        @media only screen and (max-width: 480px) {
          body, table, td, p, a, li, blockquote {
            -webkit-text-size-adjust: none !important;
          }
        }

        @media only screen and (max-width: 480px) {
          body {
            width: 100% !important;
            min-width: 100% !important;
          }
        }

        @media only screen and (max-width: 480px) {
          #template-preheader .template-image.logo {
            max-width: 121px !important;
          }
          #template-preheader .template-image-inner-block {
            padding-bottom: 7px !important;
            padding-top: 7px !important;
          }
        }

        @media only screen and (max-width: 480px) {
          #template-body-cell {
            padding-top: 10px !important;
            padding-left: 0px !important;
            padding-right: 0px !important;
          }
        }

        @media only screen and (max-width: 480px) {
          .template-image {
            width: 100% !important;
          }
        }

        @media only screen and (max-width: 480px) {
          .gray-icon-container .template-image-inner-block {
            padding-top: 5px !important;
            padding-bottom: 5px !important;
          }
          .template-image-inner-block.get-connected-card {
            padding-bottom: 8px !important;
          }
          .template-image-inner-block.get-connected-card-header {
            padding-bottom: 20px !important;
            padding-top: 0px !important;
            padding-right: 0px !important;
            padding-left: 0px !important;
          }
          .logo-description {
            padding: 5px 25px 16px !important;
          }
          .logo-description p {
            font-size: 14px !important;
          }
          .template-divider-block-inner {
            padding-left: 20px !important;
            padding-right: 20px !important;
          }
        }

     @media only screen and (max-width: 480px) {
          .logo-image {
            padding-top: 17px !important;
            padding-bottom: 5px !important;
          }
          .logo-image .template-image.logo {
            width: 77px !important;
          }
        }

        @media only screen and (max-width: 480px) {
          .template-text-container {
            max-width: 100% !important;
            width: 100% !important;
          }
          .template-image-inner-block {
            padding-left: 10px !important;
            padding-right: 10px !important;
          }
          .trainer-card .template-image-inner-block {
            padding-left: 20px !important;
            padding-right: 20px !important;
          }
          .template-text-container.columns-2 {
            max-width: 50% !important;
            width: 50% !important;
          }

        }

        @media only screen and (max-width: 480px) {
          .template-text-content {
            padding-right: 0px !important;
            padding-left: 0px !important;
            padding-bottom: 20px !important;
          }
        }
            @media only screen and (max-width: 480px) {
          .template-text-content-footer {
            padding-right: 10px !important;
            padding-left: 10px !important;
          }
        }
            @media only screen and (max-width: 480px) {
          .template-text-content-body {
            padding-right: 20px !important;
            padding-left: 20px !important;
            padding-bottom: 20px !important;
          }
        }

        @media only screen and (max-width: 480px) {
          h1 {
            font-size: 28px !important;
            line-height: 1.3 !important;
          }
        }

        @media only screen and (max-width: 480px) {
          h2 {
            font-size: 20px !important;
            line-height: 125% !important;
          }
        }

        @media only screen and (max-width: 480px) {
          h3 {
            font-size: 18px !important;
            line-height: 125% !important;
          }
        }

        @media only screen and (max-width: 480px) {
          h4 {
            font-size: 16px !important;
            line-height: 150% !important;
          }
        }

        @media only screen and (max-width: 480px) {
          #template-preheader {
            display: block !important;
          }
        }

        @media only screen and (max-width: 480px) {
          #template-preheader .template-text-content,
          #template-preheader .template-text-content p {
            font-size: 14px !important;
            line-height: 150% !important;
          }
        }

        @media only screen and (max-width: 480px) {
          #template-header .template-text-content,
          #template-header .template-text-content p {
            font-size: 16px !important;
            line-height: 150% !important;
          }
        }

        @media only screen and (max-width: 480px) {
          #template-body .template-text-content,
          #template-body .template-text-content p {
            font-size: 16px !important;
            line-height: 150% !important;
          }
        }

        @media only screen and (max-width: 480px) {
          #template-body .template-text-content-header,
          #template-body .template-text-content-header p {
            font-size: 12px !important;
            line-height: 150% !important;
          }
        }
        @media only screen and (max-width: 580px) {
          .tablet-ph-20 {
            padding-left: 40px !important;
            padding-right: 40px !important;
          }
          .column-wrapper{
            max-width:100% !important;
            width:100% !important;
          }
          .gauge-widget .column-wrapper {
            width: 33.33% !important;
            max-width: 33.33% !important;
          }
          .gauge-widget .gauge-number {
            font-size: 60px !important;
            line-height: 60px !important;
          }
          .take-ride-boxes .template-image-card-block-inner {
            padding-top: 20px !important;
            padding-bottom: 20px !important;
          }
        }
           @media only screen and (max-width: 480px) {
          .template-text-content-offer {
            padding-right: 0px !important;
            padding-left: 0px !important;
            padding-bottom: 4px !important;
            padding-top: 5px !important;
            font-size: 12px !important;
          }
        }

        @media only screen and (max-width: 480px) {
          .take-ride-boxes .template-image-card-block-inner {
            padding-top: 10px !important;
            padding-bottom: 10px !important;
          }
          .tablet-ph-20 {
            padding-left: 20px !important;
            padding-right: 20px !important;
          }
          .column-wrapper{
            max-width:100% !important;
            width:100% !important;
          }
          .gauge-widget .template-image-card-block-inner {
            padding: 15px 14px !important;
          }
          .gauge-widget .template-text-content {
            padding: 0px !important;
          }
          .gauge-widget .column-wrapper {
            width: 33.33% !important;
            max-width: 33.33% !important;
          }
          .gauge-widget .gauge-number {
            font-size: 38px !important;
            line-height: 45px !important;
          }
          .gauge-widget .gauge-title, .gauge-widget .gauge-footer {
            font-size: 13px !important;
          }
        }

        @media only screen and (max-width: 480px) {
          .template-image-card-bottom-image-content{
            padding-bottom:9px !important;
          }
        }

        @media only screen and (max-width: 480px) {
          .template-boxed-text-padding {
            padding: 20px 20px 30px !important;
          }
      }
          @media only screen and (max-width: 480px) {
          .template-boxed-text-block .template-button-inner-block {
            padding: 0px 0px 0px 10px !important;
          }
        }
              @media only screen and (max-width: 480px) {
          .template-button-inner-block {
            padding: 5px 50px 30px !important;
          }
        }
              @media only screen and (max-width: 480px) {
          .template-boxed-text-block .template-button-outer-block {
            padding-left: 0px !important;
            padding-right: 9px !important;
          }
        }
        @media only screen and (max-width: 480px) {
          .mobile-ph-20 {
            padding-left: 20px !important;
            padding-right: 20px !important;
          }
        }

        @media only screen and (max-width: 480px) {
          .mobile-ph-20 {
            padding-left: 20px !important;
            padding-right: 20px !important;
          }
          .mobile-max-w-53 {
            width: 53px !important;
            max-width: 53px !important;
          }
          .mobile-max-w-140 {
            width: 140px !important;
            max-width: 140px !important;
          }
          .text-image-title {
            height: 15px !important;
          }
          .thumbnail-image {
            min-width: 280px !important;
          }
          .mobile-ph-10 {
            padding-left: 10px !important;
            padding-right: 10px !important;
          }
          #template-body .template-text-content.mobile-font-size-14 p {
            font-size: 14px !important;
          }

          .template-text-content .footer-link {
            color: #777A81 !important;
            font-size: 13px !important;
            text-decoration: underline !important;
          }
        }
      </style>
      <!--<![endif]-->
      <!--[if !mso]><!-->

      <!--<![endif]-->
    </head>





    <body style="-ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; font-family: roboto, helvetica neue, helvetica, arial, sans-serif !important; font-size: 10px; height: 100%; margin: 0; padding: 0; width: 100%;" bgcolor="#F5F7F9"><style type="text/css">
    div.preheader 
    { display: none !important; } 
    </style>
    <div class="preheader" style="font-size: 1px; display: none !important;">Speed OSS Inc.</div>
    <style type="text/css">
    body {
    font-family: roboto, helvetica neue, helvetica, arial, sans-serif !important; font-size: 10px;
    }
    body {
    height: 100%; margin: 0; padding: 0; width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; background-color: #F5F7F9;
    }
    body {
    font-family: roboto, helvetica neue, helvetica, arial, sans-serif !important; font-size: 10px;
    }
    body {
    height: 100%; margin: 0; padding: 0; width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; background-color: #F5F7F9;
    }
    @font-face {
    font-family: 'Roboto'; font-style: normal; font-weight: 100; src: local('Roboto Thin'), local('Roboto-Thin'), url('https://fonts.gstatic.com/s/roboto/v15/Jzo62I39jc0gQRrbndN6nfesZW2xOQ-xsNqO47m55DA.ttf') format('truetype');
    }
    @font-face {
    font-family: 'Roboto'; font-style: normal; font-weight: 300; src: local('Roboto Light'), local('Roboto-Light'), url('https://fonts.gstatic.com/s/roboto/v15/Hgo13k-tfSpn0qi1SFdUfaCWcynf_cDxXwCLxiixG1c.ttf') format('truetype');
    }
    @font-face {
    font-family: 'Roboto'; font-style: normal; font-weight: 400; src: local('Roboto'), local('Roboto-Regular'), url('https://fonts.gstatic.com/s/roboto/v15/zN7GBFwfMP4uA6AR0HCoLQ.ttf') format('truetype');
    }
    @font-face {
    font-family: 'Roboto'; font-style: normal; font-weight: 700; src: local('Roboto Bold'), local('Roboto-Bold'), url('https://fonts.gstatic.com/s/roboto/v15/d-6IYplOFocCacKzxwXSOKCWcynf_cDxXwCLxiixG1c.ttf') format('truetype');
    }
    </style>
      <center>

        <!-- MAIN BODY TABLE-->
        <table align="center" border="0" cellpadding="0" cellspacing="0" height="100%" width="100%" id="body-table" class="bg-c-g-peloton" style="-ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; border-collapse: collapse; height: 100%; margin: 0; mso-table-lspace: 0; mso-table-rspace: 0; padding: 0; width: 100%;" bgcolor="#F5F7F9">
          <tbody>
            <tr>
              <td align="center" valign="top" id="template-body-cell" class="ph-0" style="border-top-width: 0; height: 100%; margin: 0; padding: 10px 0px; width: 100%;">

                <!-- TEMPLATE STARTS -->

                <!--[if gte mso 9]>
                <table align="center" border="0" cellspacing="0" cellpadding="0" width="600" style="width:600px;">
                <tr>
                <td align="center" valign="top" width="600" style="width:600px;">
                <![endif]-->
                <table border="0" cellpadding="0" cellspacing="0" width="100%" class="template-container" style="-ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; border: 0; border-collapse: collapse; max-width: 600px !important; mso-table-lspace: 0; mso-table-rspace: 0;">
                  <tbody>
                    <tr>

                      <!-- PRE HEADER SECTION, SAME COLOR AS BACKGROUND. THIS IS A GOOD SPOT TO PLACE THE LOGO -->
                      <td valign="top" id="template-preheader" style="border-bottom-width: 0; border-top-width: 0; padding-bottom: 10px; padding-top: 10px;">

                        <!-- IMAGE BLOCK -->
                        <table border="0" cellpadding="0" cellspacing="0" width="100%" class="template-image-block" style="-ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; border-collapse: collapse; min-width: 100%; mso-table-lspace: 0; mso-table-rspace: 0;">
                          <tbody class="template-image-outer-block">
                            <tr>
                              <td valign="top" class="template-image-inner-block pb-20" style="padding: 10px 30px 20px;">
                                <table align="left" width="100%" border="0" cellpadding="0" cellspacing="0" class="template-image-container" style="-ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; border-collapse: collapse; min-width: 100%; mso-table-lspace: 0; mso-table-rspace: 0;">
                                  <tbody>
                                    <tr>
                                      <td class="template-image-content" valign="top" style="padding: 0 10px;" align="center">
                                        <!-- TO CHANGE THE CONTENT OF IMAGE YOU MUST CHANGE THE src ATTRIBUTE BELOW WITH THE DESIRED URL -->
                                        <!-- IS RECOMENDED TO ADD A width ATTRIBUTE AND A max-width STYLE WITH THE WIDTH OF THE IMAGE TO BE CONSISTENT ALL OVER THE EMAIL CLIENTS -->

                                      </td>
                                    </tr>
                                  </tbody>
                                </table>
                              </td>
                            </tr>
                          </tbody>
                        </table>
                        <!-- END IMAGE BLOCK -->

                      </td>
                      <!-- END PRE HEADER SECTION -->

                    </tr>
                    <tr>

                      <!-- HEADER SECTION, GOOD SPOT TO PLACE A HEADER IMAGE -->
                      <td valign="top" id="template-header" style="border-bottom-width: 0; border-top-width: 0; padding-bottom: 0;" bgcolor="#ffffff">

                        <!-- TEXT BLOCK -->
                        <table border="0" cellpadding="0" cellspacing="0" width="100%" class="template-text-block" style="min-width: 100%; border-collapse: collapse; mso-table-lspace: 0; mso-table-rspace: 0; max-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;">
                          <tbody class="template-text-outer-block" style="max-width: 100%; min-width: 100%; border-collapse: collapse; mso-table-lspace: 0; mso-table-rspace: 0; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;">
                            <tr>
                              <td valign="top" class="template-text-inner-block" style="padding-top: 0px;">
                                <!--[if mso]>
                                        <table align="left" border="0" cellspacing="0" cellpadding="0" width="100%" style="width:100%;">
                                        <tr>
                                        <![endif]-->

                                                    <!--[if mso]>
                                        <td valign="top" width="600" style="width:600px;">
                                        <![endif]-->
                                <table align="left" border="0" cellpadding="0" cellspacing="0" width="100%" class="template-text-container" style="max-width: 100%; min-width: 100%; border-collapse: collapse; mso-table-lspace: 0; mso-table-rspace: 0; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;background-color: #ff3347;">
                                  <tbody>
                                    <tr>
                                      <td valign="top" class="template-text-content-offer" style="line-height: 1.5; word-break: break-word; color: #202020; padding: 10px 5px 8px;">
                                        <!-- TEXT CONTENT GOES HERE, IT'S RECOMMENDED TO WRAP THE TEXT INSIDE A <h1>, <h2>, <hx.... OR A <p> tag -->
                                        <p class="align-l template-text-content-offer" style="font-family: roboto, helvetica neue, helvetica, arial, sans-serif !important; margin: 0; font-size: 14px; font-weight: 300; mso-line-height-rule: exactly; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; line-height: 1.5; color: #ffffff; -moz-hyphens: none; -ms-hyphens: none; -webkit-hyphens: none; hyphens: none;letter-spacing: 1.4;" align="center">
                                      <a href="http://click.e.onepeloton.com/?qs=f036ee7ec63a4e98607b8ba4facbb50b2fb5193bc0e0066625aa8122126c7ed49ba3a43f3b6aec7b67b826e53b79d25a40fe2237e92debc6" target="_blank" style="font-weight: 700;color:#ffffff !important;text-decoration: none;">ADD A REWARD TO YOUR GITHUB ISSUE</a>
                                        </p>
                                      </td>
                                    </tr>
                                  </tbody>
                                </table>
                                <!--[if mso]>
                                        </td>
                                        <![endif]-->

                                <!--[if mso]>
                                        </tr>
                                        </table>
                                        <![endif]-->
                              </td>
                            </tr>
                          </tbody>
                        </table>
                        <!-- END TEXT BLOCK -->
                                            <!-- IMAGE BLOCK -->
                        <table border="0" cellpadding="0" cellspacing="0" width="100%" class="template-image-block bg-c-w" style="-ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; border-collapse: collapse; min-width: 100%; mso-table-lspace: 0; mso-table-rspace: 0;" bgcolor="#ffffff">
                          <tbody class="template-image-outer-block">
                            <tr>
                              <td valign="top" class="template-image-inner-block get-connected-card-header" style="padding: 0px 0px 36px;">
                                <table align="left" width="100%" border="0" cellpadding="0" cellspacing="0" class="template-image-container" style="-ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; border-collapse: collapse; min-width: 100%; mso-table-lspace: 0; mso-table-rspace: 0;">
                                  <tbody>
                                    <tr>
                                      <td class="template-image-content-header" valign="top" style="padding:0 0 0px;" align="center"> <a href="http://click.e.onepeloton.com/?qs=f036ee7ec63a4e988db02fb6fadd85d38ab4deeb00824e18327ed27b8537705a06a4c3d63d536dee603f4d69269793c33ce8895d5aaaf849"  style="text-decoration: none" target="_blank">
                                        <!-- TO CHANGE THE CONTENT OF IMAGE YOU MUST CHANGE THE src ATTRIBUTE BELOW WITH THE DESIRED URL -->
                                        <!-- IS RECOMENDED TO ADD A width ATTRIBUTE AND A max-width STYLE WITH THE WIDTH OF THE IMAGE TO BE CONSISTENT ALL OVER THE EMAIL CLIENTS -->
                                        <a href="http://click.e.onepeloton.com/?qs=f036ee7ec63a4e98607b8ba4facbb50b2fb5193bc0e0066625aa8122126c7ed49ba3a43f3b6aec7b67b826e53b79d25a40fe2237e92debc6" target="_blank"><img align="center" alt="" src="https://images.pexels.com/photos/9352/glass-time-watch-business.jpg?auto=compress&cs=tinysrgb&dpr=2&h=750&w=1260" width="600" style="width: 100%; max-width: 100%; outline: none; padding-bottom: 0; text-decoration: none; vertical-align: bottom;" class="template-image"></a>
                                      </td>
                                    </tr>
                                  </tbody>
                                </table>
                              </td>
                            </tr>
                          </tbody>
                        </table>
                        <!-- END IMAGE BLOCK -->
                                                                <!-- IMAGE BLOCK -->
                        <table border="0" cellpadding="0" cellspacing="0" width="100%" class="template-image-block bg-c-w" style="-ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; border-collapse: collapse; min-width: 100%; mso-table-lspace: 0; mso-table-rspace: 0;" bgcolor="#ffffff">
                          <tbody class="template-image-outer-block">
                            <tr>
                              <td valign="top" class="template-image-inner-block get-connected-card-header" style="padding: 0px 0px 26px;">
                                <table align="left" width="100%" border="0" cellpadding="0" cellspacing="0" class="template-image-container" style="-ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; border-collapse: collapse; min-width: 100%; mso-table-lspace: 0; mso-table-rspace: 0;">
                                  <tbody>
                                    <tr>
                                      <td class="template-image-content-header" valign="top" style="padding:0 0 0px;" align="center"> <a href="http://click.e.onepeloton.com/?qs=f036ee7ec63a4e988db02fb6fadd85d38ab4deeb00824e18327ed27b8537705a06a4c3d63d536dee603f4d69269793c33ce8895d5aaaf849"  style="text-decoration: none" target="_blank">
                                        <!-- TO CHANGE THE CONTENT OF IMAGE YOU MUST CHANGE THE src ATTRIBUTE BELOW WITH THE DESIRED URL -->
                                        <!-- IS RECOMENDED TO ADD A width ATTRIBUTE AND A max-width STYLE WITH THE WIDTH OF THE IMAGE TO BE CONSISTENT ALL OVER THE EMAIL CLIENTS -->

                                      </td>
                                    </tr>
                                  </tbody>
                                </table>
                              </td>
                            </tr>
                          </tbody>
                        </table>
                        <!-- END IMAGE BLOCK -->

                        <!-- TEXT BLOCK -->
                        <table border="0" cellpadding="0" cellspacing="0" width="100%" class="template-text-block" style="min-width: 100%; border-collapse: collapse; mso-table-lspace: 0; mso-table-rspace: 0; max-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;">
                          <tbody class="template-text-outer-block" style="max-width: 100%; min-width: 100%; border-collapse: collapse; mso-table-lspace: 0; mso-table-rspace: 0; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;">
                            <tr>
                              <td valign="top" class="template-text-inner-block" style="padding-top: 0px;">
                                <!--[if mso]>
                                        <table align="left" border="0" cellspacing="0" cellpadding="0" width="100%" style="width:100%;">
                                        <tr>
                                        <![endif]-->

                                                    <!--[if mso]>
                                        <td valign="top" width="600" style="width:600px;">
                                        <![endif]-->
                                <table align="left" border="0" cellpadding="0" cellspacing="0" width="100%" class="template-text-container" style="max-width: 100%; min-width: 100%; border-collapse: collapse; mso-table-lspace: 0; mso-table-rspace: 0; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;">
                                  <tbody>
                                    <tr>
                                      <td valign="top" class="template-text-content-body" style="line-height: 1.5; word-break: break-word; color: #ffffff; padding: 0 70px 0px;">
                                        <!-- TEXT CONTENT GOES HERE, IT'S RECOMMENDED TO WRAP THE TEXT INSIDE A <h1>, <h2>, <hx.... OR A <p> tag -->
                                        <p class="align-l" style="font-family: roboto, helvetica neue, helvetica, arial, sans-serif !important; margin: 0; font-size: 16px; font-weight: 300; mso-line-height-rule: exactly; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; line-height: 1.5; color: #181A1D; -moz-hyphens: none; -ms-hyphens: none; -webkit-hyphens: none; hyphens: none;" align="center">
                                          Hi '''.decode('utf-8')+ first_name.title()+''', we noticed you posted <a href="'''.decode('utf-8')+issue_url+'''">this</a> issue on Github. Check out Speed! You can set a reward for your issue. Your issue will be solved <strong>faster</strong> and you will help <span class="apple-link-overwrite-black2"><strong>support</strong> the open source community.</span>                                  
                                      </td>
                                    </tr>
                                  </tbody>
                                </table>
                                <!--[if mso]>
                                        </td>
                                        <![endif]-->

                                <!--[if mso]>
                                        </tr>
                                        </table>
                                        <![endif]-->
                              </td>
                            </tr>
                          </tbody>
                        </table>
                        <!-- END TEXT BLOCK -->


                                                <!-- BUTTON BLOCK -->
                                                <table border="0" cellpadding="0" cellspacing="0" width="100%" class="template-button-block" style="min-width: 100%; border-collapse: collapse; mso-table-lspace: 0; mso-table-rspace: 0; max-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;">
                                                  <tbody class="template-button-outer-block">
                                                    <tr>
                                                      <td valign="top" align="center" class="template-button-inner-block" style="padding: 30px 28px 50px;">
                                                        <table border="0" cellpadding="0" cellspacing="0" class="template-button-container" style="border-collapse: separate !important; mso-table-lspace: 0; mso-table-rspace: 0; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" bgcolor="#FF3347">
                                                          <tbody>
                                                            <tr>
                                                              <td align="center" valign="middle" class="template-button-content" style="font-family: roboto, helvetica neue, helvetica, arial, sans-serif !important; font-size: 14px; padding: 16px 45px;">
                                                                <a class="template-button" title="ADD A REWARD" href="https://www.speedoss.com" target="_blank" style="mso-line-height-rule: exactly; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; display: block; position: relative; text-decoration: none; font-weight: 900; text-align: center; color: #FFFFFF; letter-spacing: 1.4; line-height: 20px;">
                                                                  ADD A REWARD
                                                                </a>
                                                              </td>
                                                            </tr>
                                                          </tbody>
                                                        </table>
                                                      </td>
                                                    </tr>
                                                  </tbody>
                                                </table>
                                                <!-- END BUTTON BLOCK -->


                    </tr>
                    <tr>
    <!-- END BODY SECTION -->

                    </tr>
                    <tr>

                    </tr>
                    <tr>


                        <!-- FOOTER SECTION -->
                      <td valign="top" id="template-footer" style="border-bottom-width: 0; border-top-width: 0; padding-bottom: 40px; padding-top: 10px;" bgcolor="#F5F7F9">
                                                                    <!-- TEXT BLOCK -->
                        <table border="0" cellpadding="0" cellspacing="0" width="100%" class="template-text-block" style="min-width: 100%; border-collapse: collapse; mso-table-lspace: 0; mso-table-rspace: 0; max-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;">
                          <tbody class="template-text-outer-block" style="max-width: 100%; min-width: 100%; border-collapse: collapse; mso-table-lspace: 0; mso-table-rspace: 0; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;">
                            <tr>
                              <td valign="top" class="template-text-inner-block" style="padding-top: 0px;">
                                <!--[if mso]>
                                        <table align="left" border="0" cellspacing="0" cellpadding="0" width="100%" style="width:100%;">
                                        <tr>
                                        <![endif]-->

                                                    <!--[if mso]>
                                        <td valign="top" width="600" style="width:600px;">
                                        <![endif]-->
                                <table align="left" border="0" cellpadding="0" cellspacing="0" width="100%" class="template-text-container" style="max-width: 100%; min-width: 100%; border-collapse: collapse; mso-table-lspace: 0; mso-table-rspace: 0; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;">
                                  <tbody>
                                    <tr>
                                      <td valign="top" class="template-text-content-footer" style="line-height: 1.5; word-break: break-word; color: #202020; padding: 0 10px 0px;">
                                        <!-- TEXT CONTENT GOES HERE, IT'S RECOMMENDED TO WRAP THE TEXT INSIDE A <h1>, <h2>, <hx.... OR A <p> tag -->
                                        <p class="align-l" style="font-family: roboto, helvetica neue, helvetica, arial, sans-serif !important; margin: 0; font-size: 8px; font-weight: 300; mso-line-height-rule: exactly; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; line-height: 1.5; color: #777A81; -moz-hyphens: none; -ms-hyphens: none; -webkit-hyphens: none; hyphens: none;" align="left">
                                       <span class="apple-link-overwrite-black"></span>
                                        </p>
                                      </td>
                                    </tr>
                                  </tbody>
                                </table>
                                <!--[if mso]>
                                        </td>
                                        <![endif]-->

                                <!--[if mso]>
                                        </tr>
                                        </table>
                                        <![endif]-->
                              </td>
                            </tr>
                          </tbody>
                        </table>
                        <!-- END TEXT BLOCK -->



                        <!-- TEXT BLOCK -->
                        <table border="0" cellpadding="0" cellspacing="0" width="100%" class="template-text-block" style="border-collapse: collapse; min-width: 100%; mso-table-lspace: 0; mso-table-rspace: 0;">
                          <tbody class="template-text-outer-block">
                            <tr>
                              <td valign="top">
                                <!--[if mso]>
                                  <table align="center" border="0" cellspacing="0" cellpadding="0" width="100%" style="width:100%;">
                                  <tr>
                                <![endif]-->

                                <!--[if mso]>
                                  <td valign="top" width="600" style="width:600px;">
                                <![endif]-->
                                <table align="center" border="0" cellpadding="0" cellspacing="0" class="template-text-container" style="-ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; border-collapse: collapse; max-width: 100%; min-width: 100%; mso-table-lspace: 0; mso-table-rspace: 0;">
                                  <tbody>
                                    <tr>
                                      <td valign="top" class="template-text-content" style="font-family: roboto, helvetica, arial, sans-serif !important;color: #777A81; font-size: 13px; font-weight: 300; line-height: 22px; padding: 0 20px 10px; word-break: break-word;" align="center">
                                        <!-- TEXT CONTENT GOES HERE, IT'S RECOMMENDED TO WRAP THE TEXT INSIDE A <h1>, <h2>, <hx.... OR A <p> tag -->
                                        &copy; 2018 Speed OSS, Inc. <span style="white-space: nowrap;">All rights reserved.</span><br style="font-size: 0;">
                                        <span class="ns">Palo Alto, CA</span><br style="font-size: 0;">
                                        <!-- FOOTER LINKS --> 
                                        <u><a href="#" class="footer-link" style="-ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; color: #777A81; display: inline-block; font-size: 13px; mso-line-height-rule: exactly; position: relative; text-decoration: underline;"></a></u>      <u><a href="#" class="footer-link" style="-ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; color: #777A81; display: inline-block; font-size: 13px; mso-line-height-rule: exactly; position: relative; text-decoration: underline;"></a></u><br style="font-size: 0;">
                                        <u><a href="https://www.speedoss.com" class="footer-link" style="-ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; color: #777A81; display: inline-block; font-size: 13px; mso-line-height-rule: exactly; position: relative; text-decoration: underline;">Join our community</a></u>
                                      </td>
                                    </tr>
                                  </tbody>
                                </table>
                                <!--[if mso]>
                                  </td>
                                  <![endif]-->

                                <!--[if mso]>
                                  </tr>
                                  </table>
                                <![endif]-->
                              </td>
                            </tr>
                          </tbody>
                        </table>
                        <!-- END TEXT BLOCK -->

                      </td>
                      <!-- END FOOTER SECTION -->

                    </tr>
                  </tbody>
                </table>
                <!--[if gte mso 9]>
                  </td>
                  </tr>
                  </table>
                <![endif]-->
                <!-- // END TEMPLATE -->
              </td>
            </tr>
          </tbody>

        </table>
        <!-- END MAIN BODY TABLE -->
      </center>
    </body>
    </html>
    '''.decode('utf-8')

In [104]:
from __future__ import print_function
from googleapiclient.discovery import build
from apiclient import errors
from httplib2 import Http
from email.mime.text import MIMEText
import base64
from google.oauth2 import service_account

def create_message(sender, to, subject, message_text):
  """Create a message for an email.
  Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.
  Returns:
    An object containing a base64url encoded email object.
  """
  message = MIMEText(message_text, 'html')
  message['to'] = to
  message['from'] = sender
  message['subject'] = subject
  return {'raw': base64.urlsafe_b64encode(message.as_string())}

def send_message(service, user_id, message):
  """Send an email message.
  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message: Message to be sent.
  Returns:
    Sent Message.
  """
  try:
    message = (service.users().messages().send(userId=user_id, body=message)
               .execute())
    print('Message Id: %s' % message['id'])
    return message
  except errors.HttpError as error:
    print('An error occurred: %s' % error)

def service_account_login():
  SCOPES = ['https://www.googleapis.com/auth/gmail.send']
  SERVICE_ACCOUNT_FILE = 'Speed-d90429ee3804.json'

  credentials = service_account.Credentials.from_service_account_file(
          SERVICE_ACCOUNT_FILE, scopes=SCOPES)
  delegated_credentials = credentials.with_subject(EMAIL_FROM)
  service = build('gmail', 'v1', credentials=delegated_credentials)
  return service




In [149]:
def send_github_proactive_email(USER_ID, ISSUE_URL, REPO_NAME, IS_LIVE):
    resp = get(USER_ID)
    if 'data' not in resp:
        return 'NO EMAIL FOUND'
    EMAIL_TO, FIRST_NAME = get(USER_ID)['data']
    EMAIL_TO = EMAIL_TO.pop()
    print("FOUND EMAIL {} FOR USER {}".format(EMAIL_TO, USER_ID))
    if not IS_LIVE:
        EMAIL_TO = 'daljeetv@gmail.com'
    EMAIL_CONTENT = get_email_content(FIRST_NAME, ISSUE_URL)
    EMAIL_FROM = 'info@speedoss.com'
    EMAIL_SUBJECT = 'Speed OSS'
    service = service_account_login()
    # Call the Gmail API
    message = create_message(EMAIL_FROM, EMAIL_TO, EMAIL_SUBJECT, EMAIL_CONTENT)
    sent = send_message(service,'me', message)
    return EMAIL_TO

In [150]:
import pandas as pd
df = pd.read_csv('results-20190111-184936.csv')
print(df.columns)
row_iterator = df.iterrows()
start_at = 100
end_at = 120
for x in range(0, end_at+1):
    index, row = row_iterator.next()
    html_url   = row['html_url']
    repo_name  = row['name']
    url        = row['url']
    login      = row['login']
    created_at = row['created_at']
    stars      = row['stars']
    # Email variables. Modify this!
    if x >= start_at and x < end_at:
        try:
            USER_ID=login
            ISSUE_URL=html_url
            REPO_NAME=repo_name
            IS_LIVE=True
            sent_to = send_github_proactive_email(USER_ID, ISSUE_URL, REPO_NAME, IS_LIVE)
            print(sent_to)
            print(x, html_url, name, url, login, created_at, stars)
        except Exception as e:
            print("UNABLE TO SEND EMAIL: {}".format(e))

Index([u'html_url', u'name', u'url', u'login', u'created_at', u'stars',
       u'f0_'],
      dtype='object')
UNABLE TO SEND EMAIL: string indices must be integers
NO EMAIL FOUND
101 https://github.com/atom/atom/issues/18683 tensorflow/tensorflow https://api.github.com/repos/atom/atom HarsheetKakar 2019-01-11 18:00:34 UTC 37403


Traceback (most recent call last):
  File "/Users/dvirdi/src/speed/marketing/email_scraper/venv/lib/python2.7/site-packages/googleapiclient/discovery_cache/__init__.py", line 41, in autodetect
    from . import file_cache
  File "/Users/dvirdi/src/speed/marketing/email_scraper/venv/lib/python2.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    'file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth')
ImportError: file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth


FOUND EMAIL cyrilcril@gmail.com FOR USER seewhy17
Message Id: 168400998092074d
cyrilcril@gmail.com
102 https://github.com/axios/axios/issues/1956 tensorflow/tensorflow https://api.github.com/repos/axios/axios seewhy17 2019-01-11 10:44:04 UTC 36844
NO EMAIL FOUND
103 https://github.com/x64dbg/x64dbg/issues/2100 tensorflow/tensorflow https://api.github.com/repos/x64dbg/x64dbg Antiever 2019-01-11 14:46:50 UTC 36756


Traceback (most recent call last):
  File "/Users/dvirdi/src/speed/marketing/email_scraper/venv/lib/python2.7/site-packages/googleapiclient/discovery_cache/__init__.py", line 41, in autodetect
    from . import file_cache
  File "/Users/dvirdi/src/speed/marketing/email_scraper/venv/lib/python2.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    'file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth')
ImportError: file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth


FOUND EMAIL dimitri.kopriwa@yeutech.vn FOR USER kopax
Message Id: 1684009a0a6c9b9f
dimitri.kopriwa@yeutech.vn
104 https://github.com/webpack/webpack/issues/8616 tensorflow/tensorflow https://api.github.com/repos/webpack/webpack kopax 2019-01-11 16:40:46 UTC 36266
NO EMAIL FOUND
105 https://github.com/golang/go/issues/29676 tensorflow/tensorflow https://api.github.com/repos/golang/go darkfeline 2019-01-11 07:49:47 UTC 35817


...

FOUND EMAIL ryan.prichard@gmail.com FOR USER rprichard
Message Id: 168400a0c4495b8b
ryan.prichard@gmail.com
116 https://github.com/golang/go/issues/29674 tensorflow/tensorflow https://api.github.com/repos/golang/go rprichard 2019-01-11 06:48:07 UTC 35817
NO EMAIL FOUND
117 https://github.com/golang/go/issues/29670 tensorflow/tensorflow https://api.github.com/repos/golang/go cnst 2019-01-11 05:05:42 UTC 35817


Traceback (most recent call last):
  File "/Users/dvirdi/src/speed/marketing/email_scraper/venv/lib/python2.7/site-packages/googleapiclient/discovery_cache/__init__.py", line 41, in autodetect
    from . import file_cache
  File "/Users/dvirdi/src/speed/marketing/email_scraper/venv/lib/python2.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    'file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth')
ImportError: file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth


FOUND EMAIL paul@myitcv.io FOR USER myitcv
Message Id: 168400a211ded9f2
paul@myitcv.io
118 https://github.com/golang/go/issues/29687 tensorflow/tensorflow https://api.github.com/repos/golang/go myitcv 2019-01-11 16:58:30 UTC 35817


Traceback (most recent call last):
  File "/Users/dvirdi/src/speed/marketing/email_scraper/venv/lib/python2.7/site-packages/googleapiclient/discovery_cache/__init__.py", line 41, in autodetect
    from . import file_cache
  File "/Users/dvirdi/src/speed/marketing/email_scraper/venv/lib/python2.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    'file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth')
ImportError: file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth


FOUND EMAIL josharian+github@gmail.com FOR USER josharian
Message Id: 168400a26d7d23b0
josharian+github@gmail.com
119 https://github.com/golang/go/issues/29693 tensorflow/tensorflow https://api.github.com/repos/golang/go josharian 2019-01-11 19:37:08 UTC 35817
